# cpp / SQL style business card

SQL version to be added (as it is less beutiful than cpp version)

In [ ]:
# Import necessary packages
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Rectangle
from matplotlib.font_manager import FontProperties
from matplotlib.textpath import TextPath
import matplotlib.image as mpimg
from functools import lru_cache

### Transparent white pixels for QR Code

We can use this tool to set white pixels to transparent (e.g., for the QR code).

In [ ]:
from PIL import Image

def remove_white_bg(png_path, threshold=240):
    img = Image.open(png_path).convert("RGBA")
    datas = img.getdata()

    newData = []
    for item in datas:
        r, g, b, a = item
        # Set pixels close to white to transparent
        if r >= threshold and g >= threshold and b >= threshold:
            newData.append((255, 255, 255, 0))  # -> transparent
        else:
            newData.append((r, g, b, a))
    img.putdata(newData)
    return img

qr_img = remove_white_bg("QR_Code.png")
# qr_img.save("QR_Code_transparent.png")  # or: plt.imshow(qr_img)

## cpp version

In [ ]:
# Important parameters
card_w, card_h = 90, 54 # Width and Hight of the card, 90 and 54 mm
qr_path = "QR_Code.png"  # Path to the QR_code image

title = "title/name/filename.c/cpp/h" # Title illustrated on the title bar of window

save_path = "/your/path/save.pdf"

In [ ]:
bleed = 3.0
page_w, page_h = card_w + 2*bleed, card_h + 2*bleed
mm_to_in = 1/25.4
PT2MM = 25.4/72.0

qr_img = mpimg.imread(qr_path)
# qr_img = remove_white_bg(qr_path)

palette_dark_cpp = {
    "bg": "#1e1e1e",
    "panel": "#1e1e1e",
    "titlebar": "#2a2a2a",
    "text": "#d4d4d4",
    "comment": "#6a9955",
    "string": "#e3c07b",
    "keyword": "#569CD6",
    "type": "#4ec9b0",
    "ident": "#9cdcfe",
    "macro": "#c586c0",
    "brace": "#FFD700",
    "inner_brace": "#af00db",
    "url": "#4da3ff",
    "url_highlight": "#00EEFF"
}

def make_fig(palette, title="title/name/filename.c/cpp/h", save_path=None, fontsize=5, line_h=2.4):
    fig = plt.figure(figsize=(page_w*mm_to_in, page_h*mm_to_in), dpi=300)
    ax = fig.add_axes([0,0,1,1], frameon=False)
    ax.set_axis_off()
    ax.set_xlim(0, page_w)
    ax.set_ylim(0, page_h)
    ax.set_aspect('auto')

    # Background with bleed
    ax.add_patch(Rectangle((0,0), page_w, page_h, color=palette["bg"]))

    # Safe area panel
    r = 2
    ax.add_patch(FancyBboxPatch((bleed, bleed), card_w, card_h,
                                boxstyle=f"round,pad=0,rounding_size={r}",
                                linewidth=0, facecolor=palette["panel"]))

    # Title bar
    title_h = 6
    ax.add_patch(FancyBboxPatch((0, bleed + card_h-title_h), card_w+2*bleed, title_h+bleed,
                                boxstyle=f"round,pad=0,rounding_size={0}",
                                linewidth=0, facecolor=palette["titlebar"]))

    # Window dots
    for i, c in enumerate(["#ff5f57","#fdbc2e","#28c840"]):
        ax.add_patch(plt.Circle((bleed+5+i*3.2, bleed+card_h-title_h/2), 0.85, color=c))

    fp = FontProperties(family="DejaVu Sans Mono", size=5)  # file title
    ax.text(bleed+card_w/2, bleed+card_h-title_h/2, title,
            color=palette["text"], ha="center", va="center", fontproperties=fp)

    x0, y0 = bleed+4, bleed+6
    w, h = card_w-12, card_h-title_h-12

    # Code font
    fp_code = FontProperties(family="monospace", size=fontsize)
    PT2MM = 25.4/72.0
    ADV_MM = TextPath((0,0), "M", prop=fp_code).get_extents().width * PT2MM
    TRACK = 0 # Adjust the gap before each token, if needed

    @lru_cache(maxsize=None)
    def width_mm(s: str) -> float:
        if not s:
            return 0.0
        # mesureing the real width of the string
        measure_txt = s.replace(" ", "x")
        return TextPath((0,0), measure_txt, prop=fp_code).get_extents().width * PT2MM

    LETTER_SPACING = 0 # Adjust associated to each letter in token, if needed

    def advance_delta(prefix: str, token: str) -> float:
        if not token:
            return 0.0
        # Real adjust width for current token
        return (width_mm(prefix + token) - width_mm(prefix)) + LETTER_SPACING * len(token)

    def tprint(x, y, txt, color):
        ax.text(x, y, txt, color=color, ha="left", va="bottom",
                fontproperties=fp_code)
        
    def tprint_with_underline(x, y, txt, color, fp_code, hightlight=False):
        width = width_mm(txt)
        if hightlight: # hightlight background
            ax.add_patch(Rectangle((x+1,y-0.1), width-1, line_h*0.8, color=palette["titlebar"]))
        ax.text(x, y, txt, color=color, ha="left", va="bottom", fontproperties=fp_code)
        width = width_mm(txt)
        # Draw underline
        ax.plot([x+1, x + width], [y-0.1, y-0.1], color=color, linewidth=0.4)

    # tokens
    L = []
    L.append([("#ifndef ", "macro"), ("MACRO_H", "keyword")])
    L.append([("#define ", "macro"), ("MACRO_H", "keyword")])
    L.append([("#include ", "macro"), ("<string>", "string")])
    L.append([("#include ", "macro"), ("<vector>", "string")])
    L.append([])
    L.append([("namespace ", "keyword"), ("namespace_name", "ident"), (" { ", "brace"), ("// a comment if you want", "comment")])
    L.append([("    ", "text"), ("inline ", "keyword"), ("std::", "type"), ("string ", "text"),
              ("name", "ident"), (" = ", "text"), ("\"Your Name\"", "string"), (";", "text")])
    L.append([("    ", "text"), ("inline ", "keyword"), ("std::", "type"), ("string ", "text"),
              ("role", "ident"), (" = ", "text"),
              ("\"Your Role\"", "string"), (";", "text")])
    L.append([("    ", "text"), ("inline ", "keyword"), ("std::", "type"), ("string ", "text"),
              ("email", "ident"), (" = ", "text"),
              ("\"your.mail.address@mail.com\"", "string"), (";", "text")])
    L.append([("    ", "text"), ("inline ", "keyword"), ("std::", "type"), ("string ", "text"),
              ("ln", "ident"), (" = ", "text"),
              ("\"https://www.linkedin.com/in/YOUR_LINKEDIN/\"", "url_highlight"), (";", "text")])
    L.append([("    ", "text"), ("inline ", "keyword"), ("std::", "type"), ("string ", "text"),
              ("gh", "ident"), (" = ", "text"),
              ("\"https://github.com/YOUR_GITHUB/\"", "url"), (";", "text")])
    L.append([("    ", "text"), ("inline ", "keyword"), ("std::", "type"), ("vector<", "text"), ("std::", "type"), ("string> ", "text"),
              ("langs", "ident"), (" = ", "text"), ("{ ", "inner_brace"),
              ("\"EN\", ", "string"), ("\"FR\", ", "string"), ("\"ZH\"", "string"),
              (" }", "inner_brace"), (";", "text")])
    L.append([("    ", "text"), ("inline ", "keyword"), ("std::", "type"), ("string ", "text"),
              ("loc", "ident"), (" = ", "text"),
              ("Paris, France", "string"), (";", "text")])
    L.append([("    ", "text"), ("inline ", "keyword"), ("bool ", "text"),
              ("flag_relocation", "ident"), (" = ", "text"),
              ("true", "keyword"), (";", "text")])
    L.append([("} ", "brace"), ("// namespace namespace_name", "comment")])
    L.append([("#endif ", "macro"), ("// MACRO_H", "comment")])

    # draw
    y = y0 + h
    for line in L:
        y -= line_h
        x = x0
        prefix = ""                    # 累积到当前 token 之前的整段文本
        for token, key in line:
            color = palette[key]
            if "url" in key:
                if "highlight" in key:
                    tprint_with_underline(x, y, token, color, fp, True)
                else:
                    tprint_with_underline(x, y, token, color, fp)
            else:
                tprint(x, y, token, color)
            x += advance_delta(prefix, token)
            prefix += token            # 更新前缀，下一次量“前缀+新 token”的宽度差
            if prefix and prefix[-1] == ":":
                x += 0.35

    

    if save_path:
        fig.savefig("front_dark_cpp_version.pdf", dpi=300, format="pdf")

    return fig, ax

fig, ax = make_fig(palette_dark_cpp, title=title, save_path=save_path)

print("end of section")

### Add a QR Code, if needed

In [ ]:
qr_size = 20.0 # QR code size
qr_margin = 3 # Margin to the edges
qr_pad = 0 # Additional pad for QR code
corner_r = 2.0 # rounded coner for background

x0 = bleed + card_w - qr_margin - qr_size
y0 = bleed + card_h  - qr_margin - qr_size

ax.add_patch(FancyBboxPatch(
    (x0 - qr_pad, y0 - qr_pad),
    qr_size + 2*qr_pad, qr_size + 2*qr_pad,
    boxstyle=f"round,pad=0,rounding_size={corner_r}",
    linewidth=0.5, edgecolor=(1,1,1,0.4),
    facecolor=(1,1,1,0.9),
    zorder=5))

ax.imshow(
    qr_img,
    extent=[x0, x0 + qr_size, y0, y0 + qr_size],
    interpolation="nearest",
        zorder=6)

### Add a hand icon "clicking" a url, if needed

In [ ]:
hand_cur = mpimg.imread("Windows_Hand_Cursor.png")
xx, yy = 77, 17.5 # position of the icon
ax.imshow(hand_cur, extent=[xx, xx+4, yy, yy+4], zorder=6)